In [ ]:
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
def normalize(dataset):
    dataset_max = dataset.copy()

    for column in dataset_max.columns:
        dataset_max[column] = dataset_max[column] /dataset_max[column].abs().max()
    return dataset_max

In [ ]:
def load_file(fileName, inputcols,outputcol):
    dataset = pd.read_table(fileName, header=0, sep=",", encoding="unicode_escape")
    dataset = normalize(dataset)
    # print(dataset)
    data = dataset[inputcols]
    target = dataset[outputcol]

    trainingX,testX,trainingY,testY = train_test_split(data,target,test_size=0.33,random_state=43)
    
    return trainingX,testX,trainingY,testY

In [ ]:
def learn_model(trainingX, trainingY,inputNeurons, hiddenNeurons, outputNeurons, learningRate, epochs):
    

    c = 0
    classifier = None
    #Your custom implementation of neural networks will go here.

    W_input_to_hidden = np.random.rand(inputNeurons, hiddenNeurons) #A matrix where a row represents all the edges of a given input neuron
    W_hidden_to_output = np.random.rand(hiddenNeurons, outputNeurons)


    X_array = np.array(trainingX)
    Y_array  = np.array(trainingY)
    
    for e in (range(epochs)):
        i = 0
        print(f"For {epochs} epochs")
        predicted = list()
        # print(W_input_to_hidden)
        for I in tqdm(X_array): #TQDM gives us the progress bar for the loop. THis is important for debugging, as in case of large dataset, the loop may get stuck, and we wouldn't be able to tell that its is stuck without ths

            ################# Weighted Sum ##################
            Sum_hidden = I.dot(W_input_to_hidden) #The reason for using a numpy array to calculate the weighted sum is because the function performs parallelization and hence reduces the complexity

            ################# Sigmoid Activation Function  for hidden layer inp ##################
            Z = np.exp(-Sum_hidden)
            Activated1 = 1/(1+Z)

            ################# Weighted Sum ##################
            Sum_output = Activated1.dot(W_hidden_to_output)
            ################# Sigmoid activation 2 ###################
            
            Z2  = np.exp(-Sum_output)
            Output= 1/(1+Z2)

            # print(f'{I}\n{Sum_hidden}{Activated1}\n{Sum_output}{Output}')

            ################ BackwardPass ####################

            #1)Calculating the error of the output layer
            Y = Y_array[i]
            predicted.append(Output[0])
            del_out  = Output*((1-Output))*((Y-Output)) #This is Error(K) w.r.t the slides

            # 2) Calculating change in weight of hidden to output weights. THe result is in a  matrix
            delta_W_out = learningRate* (del_out).dot(np.resize(Activated1, (outputNeurons,hiddenNeurons)))   #the output should be of hodden layer
            delta_W_out = np.transpose(np.resize(delta_W_out,(outputNeurons,hiddenNeurons)))
            

            W_hidden_to_output += delta_W_out #+ W_hidden_to_output 



            #3)Calculating the error of the hidden layer
            del_hidden = Activated1*(1-Activated1)*W_hidden_to_output.dot(del_out)
            
            #4)Calculating the diff in weight
            delta_W_hidden = learningRate * (del_hidden).dot(np.resize(I, (hiddenNeurons,3)))
            delta_W_hidden = np.transpose(np.resize(delta_W_hidden, (hiddenNeurons,3))) #take the X vecto 
            
            # print("The delta_W_hidden = \n",delta_W_hidden,"\n The W input to hidden = \n " ,W_input_to_hidden)

            W_input_to_hidden += delta_W_hidden 

            i+=1
        print(f"The mean square error for epoch {e} is {computeError(trainingY, predicted)}")
    W_I_H = "W_I_H" #Weights from input to hidden
    W_H_O = "W_H_O" #Weights from hidden to output
    classifier = {W_I_H: W_input_to_hidden
                ,W_H_O: W_hidden_to_output
                }
    return classifier
    

In [ ]:
 def predict(classifier, testdata):
    # print(type(testdata))
    predicted_val=[]
    testx = np.array(testdata)
    #Your code to make predictions on test data using the learned model will go here
    for I in testx:
        ################# Weighted Sum ##################
        Sum_hidden = I.dot(classifier['W_I_H'])#I.dot(W_input_to_hidden) #The reason for using a numpy array to calculate the weighted sum is because the function performs parallelization and hence reduces the complexity

        ################# Sigmoid Activation Function  for hidden layer inp ##################
        Z = np.exp(-Sum_hidden)
        Activated1 = 1/(1+Z)

        ################# Weighted Sum ##################
        Sum_output = Activated1.dot(classifier['W_H_O'])
        ################# Sigmoid activation 2 ###################
        
        Z2  = np.exp(-Sum_output)
        Output= 1/(1+Z2)
        predicted_val.append(Output[0])
    return predicted_val

In [ ]:
def computeError(actual_class, predicted_class):
        
    MSE = -1    
    #Your code to compute MSE will go here
    
    MSE =1/2 * np.sum(np.square(actual_class-predicted_class))/len(actual_class)
    return MSE

In [ ]:
features, target = ["X1","X2","X3"],"Y"

print("Loading data.....")
trainingX,testX,trainingY,testY = load_file(r"/content/kseindex.csv", features, target) #/content/kseindex.csv

print("Learning model.....")

model = learn_model(trainingX,trainingY,3,4,1,0.8,10)
print(model)

print("Classifying test data......")      
# predictedY = classify(model, testX)     
predictedY = predict(model,testdata= testX)
print(predictedY)


print("Evaluating results.....")
print(computeError(testY,predictedY))



Loading data.....
Learning model.....
For 10 epochs



100%|██████████| 472/472 [00:00<00:00, 13940.35it/s]


The mean square error for epoch 0 is 0.014970829411264428
For 10 epochs



100%|██████████| 472/472 [00:00<00:00, 11794.67it/s]


The mean square error for epoch 1 is 0.014986152721079388
For 10 epochs


...

The mean square error for epoch 7 is 0.015012084652730044
For 10 epochs



100%|██████████| 472/472 [00:00<00:00, 6521.41it/s]


The mean square error for epoch 8 is 0.015008118861409965
For 10 epochs



100%|██████████| 472/472 [00:00<00:00, 9670.86it/s]

The mean square error for epoch 9 is 0.015003049198648172
{'W_I_H': array([[1.158957  , 0.7796016 , 0.65156807, 0.73122198],
       [0.84488592, 0.44433246, 1.02065604, 0.64577236],
       [0.94068628, 0.70977568, 0.74769598, 0.99015167]]), 'W_H_O': array([[-0.41592985],
       [ 0.30812068],
       [ 0.94881949],
       [ 0.14973606]])}
Classifying test data......
[0.6902277258951889, 0.6851944213170439, 0.6832369424392735, 0.6990924765827695, 0.6714464756073149, 0.7034598401141923, 0.6689281026923721, 0.7055513665216844, 0.6973519874742365, 0.6680180665959589, 0.6945215592698634, 0.6746597371143961, 0.6951729609028393, 0.697184135259384, 0.6832436755219262, 0.6683605109285625, 0.6827254727861698, 0.6689182027102697, 0.7028034008281308, 0.700543509765757, 0.6984328306785454, 0.7023565033819895, 0.6703747962833119, 0.6828602885743663, 0.6846303141182348, 0.699753934882478, 0.6951644352382468, 0.6993498178670864, 0.6886182596871004, 0.6829817561119934, 0.6954429424667706, 0.669394569574

### Here is just some extra evaluative work to finetune the model

In [ ]:
def printtt(testY, predictedY):
    xxx = np.array(testY)
    for i in range(len(predictedY)):
        print(predictedY[i], '\t\t', xxx[i])
        if i ==  50:
            break
        if i%10 in [1,2,3,4,6,8,7]:
            continue    
print()

In [ ]:

for epoch in [25,50,100, 500,1000,10000]:
    model = learn_model(trainingX,trainingY,3,4,1,0.9,epoch)
    # print(model)    
    # predictedY = classify(model, testX)     
    predictedY = predict(model,testdata= testX)
    # print(predictedY)
    # print(testY)

    #computeError(testY,predictedY)
    print(f"This is for epoch {epoch}")
    print(computeError(testY,predictedY))
    # printtt(testY, predictedY)
        
 

For 25 epochs



100%|██████████| 472/472 [00:00<00:00, 10136.04it/s]


The mean square error for epoch 0 is 0.015100192321916431
For 25 epochs



100%|██████████| 472/472 [00:00<00:00, 7291.27it/s]


The mean square error for epoch 1 is 0.015066195203593445
For 25 epochs


...


100%|██████████| 472/472 [00:00<00:00, 7623.48it/s]


The mean square error for epoch 3791 is 0.0005329581696607872
For 10000 epochs



100%|██████████| 472/472 [00:00<00:00, 7442.47it/s]


The mean square error for epoch 3792 is 0.0005327318281336591
For 10000 epochs



100%|██████████| 472/472 [00:00<00:00, 8161.03it/s]
